In [1]:
include("standard_hopfield.jl")
using .SH, Statistics, LinearAlgebra, Plots
using DelimitedFiles, Random

### Without simulated annealing

Now we start from a random pattern and we try to minimize the energy: we would like to end up in a stored pattern.

In [109]:
α = 0.1
N = 30
M = round(Int, N * α)
ξ = generate_patterns(M, N)
J = store(ξ)
nsweeps = 100

#success = 0
ntrials = 100

N_new = size(J)[1]
M_max = round(Int, N_new * 0.138)
M_new = 0
J_new = copy(J)
ξ_new = zeros(N_new, M_max)

for m in 1:M_max
    for _ in 1:ntrials
        σ = init_pattern(N)
        σ_rec = monte_carlo(J_new, σ; nsweeps = nsweeps, earlystop = 0, β = 10^6)
        overlaps = (ξ' * σ_rec)./ N
        if findmax(overlaps)[1] >= 0.95
            J_new -= σ_rec * σ_rec'
            J_new[diagind(J_new)] .= 0
            M_new += 1
            ξ_new[:, m] = σ_rec
            break
        end
    end
end

print(M_new)


1

In [106]:
ξ_new[:, 1] == ξ[:,2]

true

### With simulated annealing

In [118]:
α = 0.1
N = 30
M = round(Int, N * α)
ξ = generate_patterns(M, N)
J = store(ξ)
nsweeps = 100

#success = 0
ntrials = 100

N_new = size(J)[1]
M_max = round(Int, N_new * 0.138)
M_new = 0
J_new = copy(J)
ξ_new = zeros(N_new, M_max)

for m in 1:M_max
    for _ in 1:ntrials
        σ = init_pattern(N)
        σ_rec = monte_carlo(J_new, σ; nsweeps = nsweeps, earlystop = 0, β = 10^5, annealing = true)
        overlaps = (ξ' * σ_rec)./ N
        if findmax(overlaps)[1] >= 0.9
            J_new -= σ_rec * σ_rec'
            J_new[diagind(J_new)] .= 0
            M_new += 1
            ξ_new[:, m] = σ_rec
            break
        end
    end
end

print(M_new)



3

In [116]:
ξ

20×2 Matrix{Int64}:
  1   1
 -1  -1
  1  -1
 -1   1
  1   1
 -1  -1
 -1   1
  1   1
  1   1
  1   1
 -1   1
 -1   1
 -1   1
 -1  -1
  1   1
  1  -1
 -1   1
 -1   1
  1   1
 -1  -1

In [114]:
ξ_new

20×3 Matrix{Float64}:
 -1.0  -1.0  0.0
  1.0  -1.0  0.0
  1.0   1.0  0.0
 -1.0   1.0  0.0
 -1.0   1.0  0.0
 -1.0  -1.0  0.0
  1.0   1.0  0.0
  1.0  -1.0  0.0
 -1.0   1.0  0.0
  1.0  -1.0  0.0
 -1.0   1.0  0.0
 -1.0   1.0  0.0
  1.0  -1.0  0.0
 -1.0   1.0  0.0
  1.0   1.0  0.0
  1.0   1.0  0.0
  1.0   1.0  0.0
 -1.0  -1.0  0.0
 -1.0  -1.0  0.0
  1.0   1.0  0.0

function metropolis1(J, σ, β)
    N = length(σ)
    
    fliprate = 0
    for n in 1:N
        i = rand(1:N)
        ΔE = energy_variation(J, σ, i)
        
        if (ΔE < 0) || rand() < exp(-β*ΔE)
            σ[i] *= -1
            fliprate += 1
        end
    end
    return σ, fliprate/N
end

function monte_carlo1(J::AbstractMatrix, σ::AbstractVector;
        nsweeps = 100, earlystop = 0, β = 10, annealing = false)
    
    σ_rec = copy(σ)
    
    if annealing
        β_n = 10 .^ (range(-10, stop = log10(β), length = nsweeps))
        for sweep in 1:nsweeps
            #print(β_n[sweep], "\n")
            σ_rec, fliprate = metropolis1(J, σ_rec, β_n[sweep])
            fliprate <= earlystop && break
        end     
    else
        for sweep in 1:nsweeps
            σ_rec, fliprate = metropolis1(J, σ_rec, β)
            fliprate <= earlystop && break
        end
    end
    
    return σ_rec
end

100-element Vector{Float64}:
     0.01
     0.011497569953977356
     0.013219411484660288
     0.01519911082952934
     0.01747528400007684
     0.02009233002565047
     0.02310129700083159
     0.026560877829466867
     0.030538555088334154
     0.03511191734215131
     0.040370172585965536
     0.046415888336127795
     0.0533669923120631
     ⋮
  2154.4346900318847
  2477.0763559917114
  2848.035868435802
  3274.5491628777286
  3764.9358067924672
  4328.761281083057
  4977.023564332108
  5722.3676593502205
  6579.332246575683
  7564.633275546291
  8697.490026177835
 10000.0